# **Multi Feature Extraction - A Skeleton Structure for Image classification**

# Traffic Sign Board Detection

In [0]:
!git clone https://github.com/deepanrajm/machine_learning.git

In [0]:
!pip install mahotas

In [0]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from imutils import paths
import numpy as np
import argparse
import mahotas
import cv2
import pickle as cPickle
from google.colab.patches import cv2_imshow

In [0]:
def describe(image):
	# extract the mean and standard deviation from each channel of the image
	# in the HSV color space
	image = cv2.resize(image,(500,500))
	(means, stds) = cv2.meanStdDev(cv2.cvtColor(image, cv2.COLOR_BGR2HSV))
	colorStats = np.concatenate([means, stds]).flatten()

	# extract Haralick texture features
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	haralick = mahotas.features.haralick(gray).mean(axis=0)

	# return a concatenated feature vector of color statistics and Haralick
	# texture features
	return np.hstack([colorStats, haralick])

In [0]:
imagePaths = sorted(paths.list_images("machine_learning/Machine_learning/5_Sign_board_skeleton/Train"))
data = []
labels = []

In [0]:
for imagePath in imagePaths:
	# extract the label and load the image from disk
	label = imagePath.split("/")[-2]
	image = cv2.imread(imagePath)
	print (label)

	# extract features from the image, then update the list of lables and
	# features
	features = describe(image)
	print (features)
	labels.append(label)
	data.append(features)

In [0]:
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(data),
	np.array(labels), test_size=0.25, random_state=42)

In [0]:
model = RandomForestClassifier(n_estimators=20, random_state=42)

In [0]:
print("[INFO] training model...")
model.fit(trainData, trainLabels)

# evaluate the classifier
print("[INFO] evaluating...")
predictions = model.predict(testData)
print(classification_report(testLabels, predictions))

In [0]:
f = open("classifier.cPickle", "wb")
f.write(cPickle.dumps(model))
f.close()